# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: ...

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from utils import euclidean_similarity, pearson_similarity

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [2]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified in self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.metric = metric
        self.n_recommendations = n_recommendations
        self.alpha = alpha

    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        self.X = X
        if self.metric == 'euclidean':
            self.used_metric = lambda x, y : 1 - euclidean_similarity(x, y)
        if self.metric == 'pearson':
            self.used_metric = lambda x, y : (1 - pearson_similarity(x, y)) / 2
        self.model = NearestNeighbors(metric=self.used_metric, radius=1-self.alpha)
        self.model.fit(self.X)
        
        return self
        
    def __find_closest_users(self, user_id: int, n_closest_users: int = None):
        
        if n_closest_users is None:
            n_closest_users = self.X.shape[0]

        distances, similar_users = self.model.radius_neighbors([self.X[user_id]], return_distance=True)
        similar_users = similar_users[0][distances[0].argsort()]
        return similar_users[similar_users != user_id][:n_closest_users]

    def make_recommendation(self, user_id: int, n_closest_users: int = None, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        if n_recommendations is None:
            n_recommendations = self.n_recommendations
        ind = self.__find_closest_users(user_id=user_id, n_closest_users=n_closest_users)
        X_users = self.X[ind]
        popularity = (X_users > 0).mean(axis=0).argsort()
        
        return popularity[::-1][:n_recommendations]

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [3]:
X = np.array([[0, 0.1, 0.3],
             [0, 2, 1.5],
             [0, 0.6, 0.5],
             [0, 0.5, 0],
             [0, 1.6, 1.05]])

In [4]:
model = UserBasedRecommendation(alpha=0.1).fit(X)
model.make_recommendation(user_id=2, n_recommendations=1, n_closest_users=2)

array([1], dtype=int64)

In [5]:
model = UserBasedRecommendation(alpha=0, metric='pearson').fit(X)
model.make_recommendation(user_id=2, n_recommendations=1, n_closest_users=2)

array([2], dtype=int64)

<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv('ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [8]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [9]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [10]:
data = data.sort_values(by=['userId', 'movieId'])
data.userId = data.userId - 1
maple = {k:i for i, k in enumerate(sorted(data.movieId.unique()))}
data.movieId = data.movieId.map(maple)
data.head()

,userId,movieId,rating,timestamp
0,0,30,2.5,1260759144
1,0,833,3.0,1260759179
2,0,859,3.0,1260759182
3,0,906,2.0,1260759185
4,0,931,4.0,1260759205


In [11]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [12]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

Удалим для наиболее активных пользователей 5 оценок

In [13]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

In [14]:
test_data.head()

,userId,movieId,rating,timestamp
0,546,3340,4.0,1028129718
1,546,772,4.0,1039880724
2,546,8781,4.5,1468681977
3,546,5835,4.0,1242992741
4,546,1369,3.5,1342849917


In [15]:
test_actual = np.array([test_data.movieId[test_data.userId == user] for user in test_data.userId.unique()])

In [16]:
test_data.userId.unique()

array([546, 563, 623, 14, 72, 451, 467, 379, 310, 29], dtype=object)

In [17]:
active_users

Int64Index([546, 563, 623, 14, 72, 451, 467, 379, 310, 29], dtype='int64')

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [18]:
from scipy.sparse import csr_matrix

In [19]:
data_matrix = csr_matrix((data.rating, (data.userId, data.movieId)))
data_matrix[0, 30]

2.5

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [52]:
model = UserBasedRecommendation(alpha=0.1).fit(data_matrix.toarray())
predicted = np.empty([0,100])
for user_id in active_users:
    prediction = model.make_recommendation(user_id=user_id, n_recommendations=100)
    predicted = np.append(predicted, [prediction], axis=0)

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [33]:
from utils import mapk

In [53]:
for k in (5, 10, 100):
    print(round(mapk(test_actual, predicted, k),3))

0.024
0.024
0.026


Как можно улучшить работу модели?

<b>Ответ: Учет рейтинга фильмов у схожих пользователей; ансамбль с другими моделями или той же моделью на других параметрах. </b>